# Degradation Module

In [2]:
## Importing Packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [1]:
## Pre-processing cycle dataframe for the battery pack in question
def deg_preprocessing(df):
    df['avg_ch_C'] = (df["avg_ch_MW"]*1000)/design_dict['tot_cap'] # Charge C rate
    df['avg_disch_C'] = (df["avg_disch_MW"]*1000)/design_dict['tot_cap'] # Discharge C rate
    df['av_temp'] = [25]*len(df['cycle_num'].tolist())
    df['SOC'] = [0]*len(df['cycle_num'].tolist())
    df['max_SOC'] = [0]*len(df['cycle_num'].tolist())
    df['min_SOC'] = [0]*len(df['cycle_num'].tolist())
    
    ## Iterating through cycle dataframe
    for index,row in df.iterrows():
        if index == 0: # first row exception
            start_SOC = 0 ##start at 0 SOC
            df.loc[df['cycle_num']==row['cycle_num'],'max_SOC'] = -row['ch_th_kWh']
            df.loc[df['cycle_num']==row['cycle_num'],'min_SOC'] = start_SOC
            df.loc[df['cycle_num']==row['cycle_num'],'SOC'] = -row['ch_th_kWh']-row['disch_th_kWh']
            prev_row = df.loc[df['cycle_num']==row['cycle_num'],:]
        else:
            df.loc[df['cycle_num']==row['cycle_num'],'max_SOC'] = -row['ch_th_kWh']+prev_row['SOC'].values
            df.loc[df['cycle_num']==row['cycle_num'],'min_SOC'] = min((-row['ch_th_kWh']-row['disch_th_kWh']),
                                                                      prev_row['SOC'].values)
            df.loc[df['cycle_num']==row['cycle_num'],'SOC'] = -row['ch_th_kWh']-row['disch_th_kWh']+prev_row['SOC'].values
            prev_row = df.loc[df['cycle_num']==row['cycle_num'],:]
    ## Determining the start SOC so SOC never goes <0
    overlap = min(df['SOC'].tolist()) # should be negative
    df['min_SOC']= 100*(df['min_SOC'] - overlap)/design_dict['tot_cap']
    df['max_SOC']= 100*(df['max_SOC'] - overlap)/design_dict['tot_cap']
    df['SOC']= 100*(df['SOC'] - overlap)/design_dict['tot_cap']

    df['DOD'] = df['max_SOC'] - df['SOC']

    return df

In [ ]:
# ## Function to graph cell dispatch
# def graphCellDispatch(res_path, df):
#     fig, axs = plt.subplots(2,2, figsize =(10,6))

#     axs[0,0].plot(df['cycle_num'],df['SOC'], label = 'SOC', c='turquoise')
#     axs[0,0].plot(df['cycle_num'],df['min_SOC'], label = 'min SOC', c= 'powderblue')
#     axs[0,0].plot(df['cycle_num'],df['max_SOC'], label = 'max SOC', c= 'powderblue')
#     axs[0,0].set_ylabel('end of cycle SOC (%)')
#     axs[0,0].set_xlabel('Cycle Number')
#     axs[0,0].legend()

#     axs[0,1].plot(df['cycle_num'],df['DOD'], label = 'DOD', c='turquoise')
#     axs[0,1].set_ylabel('Depth of Discharge (%)')
#     axs[0,1].set_xlabel('Cycle Number')
#     axs[0,1].legend()

#     axs[1,0].plot(df['cycle_num'],df['avg_disch_C'], label = 'Discharge', c='turquoise')
#     axs[1,0].plot(df['cycle_num'],df['avg_ch_C'], label = 'Charge', c='red')
#     axs[1,0].set_ylabel('C rate')
#     axs[1,0].set_xlabel('Cycle Number')
#     axs[1,0].legend()


#     axs[1,1].plot(df['cycle_num'],df['av_temp'], label = 'Average', c='turquoise')
#     axs[1,1].set_ylabel('Cell Temperature (degC)')
#     axs[1,1].set_xlabel('Cycle Number')
#     axs[1,1].legend()
    
#     fig.savefig(res_path/'02_Dispatch Data'/f'{today}_Cell Dispatch_{red_factor_th}_{minimum_voltage}.png')
# #         
#     return

In [ ]:
def getNextHigher(num, li):
    higher = []
    for number in li:
        if number>num:
            higher.append(number)
            
        if higher:
            lowest = sorted(higher)[0]
            return lowest
        else: ## for when C rate is higher than all  listed
            return 0

In [6]:
def getNextLower(num, li):
    lower = []
    for number in li:
        if number<num:
            lower.append(number)
            
        if lower:
            highest = sorted(lower, reverse = True)[0]
            return lowest
        else: ## for when C rate is higher than all  listed
            return 0

In [8]:
## Function that carries out linear interpolution
def linInterp(low_a,high_a,low_p,high_p,p): 
    # a = actual data being interpolated, p = data that determines interpolation coefficient
    result = low_a + (high_a-low_a)*((p-low_p)/(high_p-low_p))
    ## In the boundary cases:
    if low_p == high_p:
        if p<low_p:
            result = low_a
        else:
            result=high_a
    return result

In [4]:
# Empirical Model Coefficients
empiricalModelCoeffs = {}

## FOR LFP (from paper: https://www.sciencedirect.com/science/article/abs/pii/S0378775310021269)
Coef = pd.DataFrame()
Coef['C-rate'] = [0.5,2,6,10]
Coef['B'] = [30330, 19330, 12000, 11500]
Coef['Ea']=[31500, 31000, 29500, 28000]
Coef['z']=[0.552, 0.554,0.56,0.56]
empiricalModelCoeffs['LFP'] = Coef

## FOR NMC (estimate)
Coef = pd.DataFrame()
Coef['C-rate'] = [0.5,2,6,10]
Coef['B'] = [40000, 30000, 25000, 20000]
Coef['Ea']=[30000, 29500, 29000, 28000]
Coef['z']=[ 0.552, 0.554,0.56,0.56]
empiricalModelCoeffs['NMC'] = Coef

## FOR NCA (estimate)
# (short cycle life, high energy density)
Coef = pd.DataFrame()
Coef['C-rate'] = [0.5,2,6,10]
Coef['B'] = [60660, 38000, 25000, 23000]
Coef['Ea']=[29000, 29000, 28000, 28000]
Coef['z']=[0.6, 0.62,0.64,0.65]
empiricalModelCoeffs['NCA'] = Coef

## FOR LCO (estimate)
Coef = pd.DataFrame()
Coef['C-rate'] = [0.5,2,6,10]
Coef['B'] = [50000, 40000, 30000, 25000]
Coef['Ea']=[31500, 31000, 29500, 28000]
Coef['z']=[0.57, 0.572,0.58,0.58]
empiricalModelCoeffs['LCO'] = Coef

## FOR IMR (estimate)
Coef = pd.DataFrame()
Coef['C-rate'] = [0.5,2,6,10]
Coef['B'] = [50500, 45000, 30000, 30000]
Coef['Ea']=[31500, 31000, 29500, 28000]
Coef['z']=[0.552, 0.554,0.56,0.56]
empiricalModelCoeffs['IMR'] = Coef

## FOR LTO (estimate)
Coef = pd.DataFrame()
Coef['C-rate'] = [0.5,2,6,10]
Coef['B'] = [15000, 10000, 6000, 5000]
Coef['Ea']=[31500, 31000, 29500, 28000]
Coef['z']=[0.552, 0.554,0.56,0.56]
empiricalModelCoeffs['LTO'] = Coef

{'lfp':    C-rate      B     Ea      z
0     0.5  30330  31500  0.552
1     2.0  19330  31000  0.554
2     6.0  12000  29500  0.560
3    10.0  11500  28000  0.560}


In [1]:
## Function to carry out degradation modelling from processed data
def empiricalDegModel(df,design_dict, EOL, chem,chem_nom_voltage, chem_nom_cap):
    ## empirical model#
    tot_dur = (df.iloc[[-1]]['time']+df.iloc[[-1]]['disch_dur']+df.iloc[[-1]]['ch_dur']+df.iloc[[-1]]['rest_dur']).values[0]
    t_df = df[['cycle_num','time','disch_dur','ch_dur','rest_dur','disch_th_kWh','ch_th_kWh','av_temp','avg_ch_C']]
    t_df['avg_ch_C'] = abs(t_df['avg_ch_C']) #making all C rates +ive
    getNearest = lambda num,collection:min(collection,key=lambda x:abs(x-num))
    
    # Empirical Model Coefficients: dependant on chemistry
    Coef = pd.DataFrame()
    Coef = empiricalModelCoeffs[chem]
    
    # Initial Conditions
    R = 8.3144626 # gas constant
    running_deg = 0
    running_SOH = 100
    
    for index, row in t_df.iterrows():
        
        ##Getting Boundary C rates to interpolate coefficients: capped by highest and lowest val (no exterpolation)
        highC = getNextHigher(row['avg_ch_C'],Coef['C-rate'].tolist())
        if highC == 0:
            highC = max(Coef['C-rate'].tolist())
            lowC = highC
        else:
            lowC = getNextLower(row['avg_ch_C'],Coef['C-rate'].tolist())
            if lowC == 0:
                lowC = min(Coef['C-rate'].tolist())
        
        t_df.loc[t_df['cycle_num']==row['cycle_num'],'avg_ch_C'] = row['avg_ch_C']

        ## Interpolating Degradation Coefficients (for relevant C rate)
        B = linInterp(Coef.loc[Coef['C-rate']==lowC,'B'].values[0],
                      Coef.loc[Coef['C-rate']==highC,'B'].values[0], 
                     lowC, highC, row['avg_ch_C'])
        Ea = linInterp(Coef.loc[Coef['C-rate']==lowC,'Ea'].values[0],
                              Coef.loc[Coef['C-rate']==highC,'Ea'].values[0], 
                             lowC, highC, row['avg_ch_C'])
        z = linInterp(Coef.loc[Coef['C-rate']==lowC,'z'].values[0],
                              Coef.loc[Coef['C-rate']==highC,'z'].values[0], 
                             lowC, highC, row['avg_ch_C'])
        
        av_th = (row['disch_th_kWh']+(-row['ch_th_kWh']))/2
        Ah = ((av_th*1000)/(chem_nom_voltage*design_dict['tot_cells']))*(2/chem_nom_cap)
        # Ah throughput per cell: (average of charge and discharge)
        # equivalent to throughput through a 2.2 Ah LFP (derated to 2Ah) with same no. of cycles
        
        temp = row['av_temp']+273.15 # Temperature (K)
        Qloss = B*(math.exp(-(Ea)/(R*temp)))*(Ah**z)# % capacity loss
        t_df.loc[t_df['cycle_num']==row['cycle_num'],'Qloss'] = Qloss
        
        running_SOH = running_SOH*((100-(Qloss))/100)
        running_deg = 100 - running_SOH
        t_df.loc[t_df['cycle_num']==row['cycle_num'],'running_deg'] = running_deg
    
    end_SOH = running_SOH
    
    ## Calculation of degradation life from EoL condition and running SoH
    deg_life = tot_dur*((np.log(EOL/100)/(np.log(end_SOH/100))))
    
    ## Storing data into dictionary
    deg_dict['test_dur'] = tot_dur
    deg_dict['end_SOH'] = end_SOH
    deg_dict['deg_life'] = deg_life
    
    return deg_dict